# College Buddy - Google Colab Setup

Run the cells below to set up the environment and chat with the AI assistant.

## 1. Setup Environment

In [ ]:
# Clone the repository
!git clone https://github.com/VijayKiran-2004/college-buddy.git

# Move into the directory
%cd college-buddy

# Install Python dependencies
!pip install -r requirements.txt
!pip install flask pyngrok

# Install Ollama (Linux version for Colab)
!curl -fsSL https://ollama.com/install.sh | sh

## 2. Start Ollama Service

In [ ]:
import subprocess
import time

# Start Ollama in the background
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("Starting Ollama service...")
time.sleep(5)  # Give it a moment to start

## 3. Pull Model

In [ ]:
!ollama pull gemma3:1b

## 4. Run Flask Server with ngrok

In [ ]:
import getpass
import os
from pyngrok import ngrok, conf

print("Enter your ngrok authtoken (from https://dashboard.ngrok.com/get-started/your-authtoken):")
conf.get_default().auth_token = getpass.getpass()

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

In [ ]:
from flask import Flask, request, jsonify
import sys

# Ensure we are in the correct directory
if os.getcwd().split('/')[-1] != 'college-buddy':
    %cd college-buddy

# Add current directory to path so imports work
sys.path.append(os.getcwd())

from app.services.rag_system import RAGSystem

print("Initializing RAG System... (this may take a minute)")
rag = RAGSystem()
print("System Ready!")

app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    query = data.get('query')
    if not query:
        return jsonify({'error': 'No query provided'}), 400
    
    try:
        response = rag(query)
        return jsonify({'response': response})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

print(f"Running Flask app on port 5000... Access via {public_url}/chat")
app.run(port=5000)